# Optimizing Prompts with **Automatic Prompt Engineer** (APE)

This notebook demonstrates how to use Automatic Prompt Engineer (APE) (arxiv link) to optimize prompts for text generation. In its simplest form, APE takes as input a dataset (a list of inputs and a list of outputs), a prompt template, and optimizes this prompt template so that it generates the outputs given the inputs.

APE accomplishes this in two steps. First, it uses a language model to generate a set of candidate prompts. Then, it uses a prompt evaluation function to evaluate the quality of each candidate prompt. Finally, it returns the prompt with the highest evaluation score.

In [1]:
# First, let's define a simple dataset consisting of words and their antonyms.
words = ["sane", "direct", "informally", "unpopular", "subtractive", "nonresidential",
    "inexact", "uptown", "incomparable", "powerful", "gaseous", "evenly", "formality",
    "deliberately", "off"]
antonyms = ["insane", "indirect", "formally", "popular", "additive", "residential",
    "exact", "downtown", "comparable", "powerless", "solid", "unevenly", "informality",
    "accidentally", "on"]

In [2]:
# Now, we need to define the format of the prompt that we are using.

eval_template = \
"""Instruction: [PROMPT]
Input: [INPUT]
Output: [OUTPUT]"""

In [3]:
# Now, let's use APE to find prompts that generate antonyms for each word.
from automatic_prompt_engineer import ape

result, demo_fn = ape.simple_ape(
    dataset=(words, antonyms),
    eval_template=eval_template,
    eval_model="facebook/opt-1.3b",
    prompt_gen_model="facebook/opt-1.3b",
    prompt_gen_mode="opt",
    prompt_gen_batch_size=100,
    eval_batch_size=250
)

Config before update: {'generation': {'num_subsamples': 5, 'num_demos': 5, 'num_prompts_per_subsample': 10, 'model': {'name': 'OPT', 'batch_size': 100, 'gpt_config': {'model': 'facebook/opt-1.3b', 'temperature': 0.9, 'max_tokens': 50, 'top_p': 0.9, 'frequency_penalty': 0.0, 'presence_penalty': 0.0}}}, 'evaluation': {'method': 'bandits', 'rounds': 20, 'num_prompts_per_round': 0.334, 'bandit_method': 'ucb', 'bandit_config': {'c': 1.0}, 'base_eval_method': 'likelihood', 'base_eval_config': {'num_samples': 5, 'num_few_shot': 5, 'model': {'name': 'GPT_forward', 'batch_size': 250, 'gpt_config': {'model': 'facebook/opt-1.3b', 'temperature': 0.7, 'max_tokens': 200, 'top_p': 1.0, 'frequency_penalty': 0.0, 'presence_penalty': 0.0}}}}, 'demo': {'model': {'name': 'GPT_forward', 'batch_size': 500, 'gpt_config': {'model': 'text-davinci-002', 'temperature': 0.7, 'max_tokens': 200, 'top_p': 1.0, 'frequency_penalty': 0.0, 'presence_penalty': 0.0}}}}
Config after update: {'generation': {'num_subsamples'

  0%|                                                                                                          | 0/1 [00:00<?, ?it/s]

<generator object OPT.__generate_text.<locals>.<genexpr> at 0x7f807fe2ef20>


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [03:32<00:00, 212.36s/it]


Model returned 10 prompts. Deduplicating...
Model: {'name': 'GPT_forward', 'batch_size': 500, 'gpt_config': {'model': 'text-davinci-002', 'temperature': 0.7, 'max_tokens': 200, 'top_p': 1.0, 'frequency_penalty': 0.0, 'presence_penalty': 0.0}}
Deduplicated to 10 prompts.
Evaluating prompts...
Model: {'name': 'OPT', 'batch_size': 500, 'gpt_config': {'model': 'facebook/opt-1.3b', 'temperature': 0.7, 'max_tokens': 200, 'top_p': 1.0, 'frequency_penalty': 0.0, 'presence_penalty': 0.0}}


Evaluating prompts:   0%|                                                                                     | 0/20 [00:17<?, ?it/s]


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [ ]:
# Let's see the results.
print(result)

Let's compare with a prompt written by a human:

"*Write an antonym to the following word.*"

In [ ]:
from automatic_prompt_engineer import ape

manual_prompt = "Write an antonym to the following word."

human_result = ape.simple_eval(
    dataset=(words, antonyms),
    eval_template=eval_template,
    prompts=[manual_prompt],
    eval_model="facebook/opt-2.7b"
)

In [ ]:
print(human_result)